In [2]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
project_root = Path.cwd().parent.parent  # Assuming notebook is in experiments/notebooks/
sys.path.insert(0, str(project_root))
print(sys.path)
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from experiments.analysis_utils.data_processing import read_directory
from experiments.analysis_utils.visualization import plot_reaction_speed_distribution, plot_token_distribution

['/home/rotsker/PycharmProjects/dg-research-revision', '/home/rotsker/miniconda3/envs/radCAD/lib/python312.zip', '/home/rotsker/miniconda3/envs/radCAD/lib/python3.12', '/home/rotsker/miniconda3/envs/radCAD/lib/python3.12/lib-dynload', '', '/home/rotsker/miniconda3/envs/radCAD/lib/python3.12/site-packages']


# Cluster A

In [8]:
path_to_runs = Path("../../results/simulations/clusterA_future_proof/")
proposal_df_full, start_data_df_full, timestep_data_df_full = read_directory(path_to_runs.resolve(), pass_directory_name=True)
for df in [proposal_df_full, start_data_df_full, timestep_data_df_full]:
    df["wallet_distribution"] = df["directory_name"].str.split("_").str[0]
    df["ModeledReactions"] = df["directory_name"].str.split("_").str[1]


In [9]:
from experiments.analysis_utils.metrics import analyze_veto_timing_by_seals
veto_stats = analyze_veto_timing_by_seals(timestep_data_df_full, start_data_df_full, additional_columns=('wallet_distribution', 'ModeledReactions'))
print("Veto statistics for different wallet distributions and ModeledReactions:")
display(veto_stats)

Veto statistics for different wallet distributions and ModeledReactions:


veto_rate  \
first_seal_rage_quit_support second_seal_rage_quit_support wallet_distribution ModeledReactions              
0.01                         0.1                           centralized         accelerated           100.0   
                                                                               normal                 91.0   
                                                                               slowed                  0.0   
                                                           decentralized       accelerated           100.0   
                                                                               normal                100.0   
                                                                               slowed                100.0   
                                                           original            accelerated           100.0   
                                                                               normal                100.0   
                                                                               slowed                 98.0   

                                                                                                 mean_time_to_veto  \
first_seal_rage_quit_support second_seal_rage_quit_support wallet_distribution ModeledReactions                      
0.01                         0.1                           centralized         accelerated               26.970000   
                                                                               normal                    33.087912   
                                                                               slowed                          NaN   
                                                           decentralized       accelerated                7.480000   
                                                                               normal                    21.180000   
                                                                               slowed                    23.700000   
                                                           original            accelerated               22.320000   
                                                                               normal                    26.000000   
                                                                               slowed                    31.561224   

                                                                                                 median_time_to_veto  \
first_seal_rage_quit_support second_seal_rage_quit_support wallet_distribution ModeledReactions                        
0.01                         0.1                           centralized         accelerated                      27.0   
                                                                               normal                           33.0   
                                                                               slowed                            NaN   
                                                           decentralized       accelerated                       7.0   
                                                                               normal                           21.0   
                                                                               slowed                           24.0   
                                                           original            accelerated                      22.0   
                                                                               normal                           26.0   
                                                                               slowed                           31.0   

                                                                                                 hours_mean_time_to_veto_hours  \
first_seal_rage_quit_support second_seal_rage_quit_support wallet_distribution ModeledReactions                                  
0.01  

In [12]:
timestep_data_df_full_with_params = timestep_data_df_full.merge(start_data_df_full, on='run_id', how='left')
threshold_times = timestep_data_df_full_with_params.groupby('run_id').apply(lambda x: x[x['actors_total_locked_relative'] >= x['second_seal_rage_quit_support']]['timestep'].min()).reset_index(name='threshold_timestep')
threshold_times['threshold_hours'] = threshold_times['threshold_timestep'] * 3
threshold_times = threshold_times.merge(start_data_df_full, on='run_id', how='left')

print("RageQuit threshold times for different wallet distributions and ModeledReactions:")
display(threshold_times.groupby(['first_seal_rage_quit_support', 'second_seal_rage_quit_support', 'wallet_distribution', 'ModeledReactions']).agg({'threshold_hours': ['median', 'min', 'max'], 'run_id': 'count'}))


RageQuit threshold times for different wallet distributions and ModeledReactions:


/tmp/ipykernel_17281/1027194245.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  threshold_times = timestep_data_df_full_with_params.groupby('run_id').apply(lambda x: x[x['actors_total_locked_relative'] >= x['second_seal_rage_quit_support']]['timestep'].min()).reset_index(name='threshold_timestep')


threshold_hours  \
                                                                                                         median   
first_seal_rage_quit_support second_seal_rage_quit_support wallet_distribution ModeledReactions                   
0.01                         0.1                           centralized         accelerated                234.0   
                                                                               normal                     237.0   
                                                                               slowed                       NaN   
                                                           decentralized       accelerated                198.0   
                                                                               normal                     204.0   
                                                                               slowed                     207.0   
                                                           original            accelerated                219.0   
                                                                               normal                     219.0   
                                                                               slowed                     222.0   

                                                                                                        \
                                                                                                   min   
first_seal_rage_quit_support second_seal_rage_quit_support wallet_distribution ModeledReactions          
0.01                         0.1                           centralized         accelerated       222.0   
                                                                               normal            222.0   
                                                                               slowed              NaN   
                                                           decentralized       accelerated       189.0   
                                                                               normal            195.0   
                                                                               slowed            201.0   
                                                           original            accelerated       204.0   
                                                                               normal            204.0   
                                                                               slowed            207.0   

                                                                                                        \
                                                                                                   max   
first_seal_rage_quit_support second_seal_rage_quit_support wallet_distribution ModeledReactions          
0.01                         0.1                           centralized         accelerated       249.0   
                                                                               normal            249.0   
                                                                               slowed              NaN   
                                                           decentralized       accelerated       207.0   
                                                                               normal            210.0   
                                                                               slowed            213.0   
                                                           original            accelerated       228.0   
                                                                               normal            228.0   
                                                                               slowed            231.0   

                                                                                                run_id  
                                                                

The only problem is that I've ran the simulations where the proposal gets executed, so we don't have the data for slow centralized distributions. Later I can rerun this particular case to get the data. Also normal centralized and slowed original.

# Larger decentralization 1. (private wallets get richer, number of wallets unchanged)

## Cluster B

In [1]:
# Read the data
path_to_runs = Path("../../results/simulations/withdrawal_queue_replacement_decentralized/")
print(path_to_runs.resolve())
print(path_to_runs.resolve().exists())
proposal_df_full, start_data_df_full, timestep_data_df_full = read_directory(path_to_runs.resolve())




NameError: name 'Path' is not defined

In [78]:
timestep_data_df_full_with_params = timestep_data_df_full.merge(start_data_df_full, on='run_id')

In [85]:
from experiments.analysis_utils.metrics import analyze_veto_timing_by_seals

veto_stats = analyze_veto_timing_by_seals(timestep_data_df_full, start_data_df_full, additional_columns=('attacker_share',))
veto_stats['attack_success_rate'] = 100 - veto_stats['veto_rate']
veto_stats['n_successful_attacks'] = veto_stats['attack_success_rate'] / 100 * veto_stats['total_runs']
veto_stats['pool_share_added_by_attacker'] = veto_stats.index.get_level_values('attacker_share').map(lambda x: x / (1 - x))
print("Veto statistics by seal parameters:")
display(veto_stats)

Veto statistics by seal parameters:


veto_rate  \
first_seal_rage_quit_support second_seal_rage_quit_support attacker_share               
0.01                         0.1                           0.250019        100.000000   
                                                           0.300021        100.000000   
                                                           0.350023        100.000000   
                                                           0.400024        100.000000   
                                                           0.450025        100.000000   
                                                           0.500025        100.000000   
                                                           0.550025        100.000000   
                                                           0.600024        100.000000   
                                                           0.700021        100.000000   
                                                           0.720020         94.339623   
                                                           0.750019         49.494949   
                                                           0.770018          6.060606   
                                                           0.800016          0.000000   
                                                           0.900009          0.000000   

                                                                           mean_time_to_veto  \
first_seal_rage_quit_support second_seal_rage_quit_support attacker_share                      
0.01                         0.1                           0.250019                22.765217   
                                                           0.300021                23.182609   
                                                           0.350023                23.540816   
                                                           0.400024                24.040816   
                                                           0.450025                24.591837   
                                                           0.500025                25.357143   
                                                           0.550025                26.265306   
                                                           0.600024                27.280000   
                                                           0.700021                31.710000   
                                                           0.720020                33.570000   
                                                           0.750019                36.530612   
                                                           0.770018                37.000000   
                                                           0.800016                      NaN   
                                                           0.900009                      NaN   

                                                                           median_time_to_veto  \
first_seal_rage_quit_support second_seal_rage_quit_support attacker_share                        
0.01                         0.1                           0.250019                       23.0   
                                                           0.300021                       23.0   
                                                           0.350023                       23.5   
                                                           0.400024                       24.0   
                                                           0.450025                       25.0   
                                                           0.500025                       25.0   
                                                           0.550025                       26.0   
                                                           0.600024                       27.0   
                                                           0.700021                       31.0   
                                    

We can see that the costs of the attack by diluting the total stETH pool are even higher (the attacker needs to add 3 pools for a 50% chance of successful attack).

## Cluster C. Bribing

In [99]:
path_to_runs = Path("../../results/simulations/bribed_withdrawal_queue_replacement/")
print(path_to_runs.resolve())
print(path_to_runs.resolve().exists())
proposal_df_full, start_data_df_full, timestep_data_df_full = read_directory(path_to_runs.resolve())
print(len(start_data_df_full))

/home/rotsker/PycharmProjects/dg-research-revision/results/simulations/bribed_withdrawal_queue_replacement
True
500


In [100]:
start_data_df_full['early_responders_count'] = start_data_df_full.quick_and_normal_early_responders_count
start_data_df_full['early_responders_tokens'] = start_data_df_full.quick_and_normal_early_responders_stETH_funds + start_data_df_full.quick_and_normal_early_responders_wstETH_funds
total_funds = timestep_data_df_full.groupby('run_id').actors_total_balance.first()
start_data_df_full['early_responders_tokens_relative'] = (
    start_data_df_full['early_responders_tokens'] / 
    start_data_df_full['run_id'].map(total_funds))
start_data_df_full['amount_needed_to_bribe'] = start_data_df_full['early_responders_tokens_relative'] - 0.01

agg_results = start_data_df_full.agg(
    {
        'early_responders_count': ('median', 'min', 'max'),
        'early_responders_tokens_relative': ('median', 'min', 'max'),
        'amount_needed_to_bribe': ('median', 'min', 'max')
    }
)
agg_results = agg_results.clip(lower=0)
agg_results

,early_responders_count,early_responders_tokens_relative,amount_needed_to_bribe
median,237.0,0.039612,0.029612
min,201.0,0.032747,0.022747
max,288.0,0.047873,0.037873


We can see that bribing amount is much higher than in case of original token distribution. It is even higher than with fast reactions.